# Pandas

Pandas is Python's answer to R.  It's a good tool for small(ish) data analysis -- i.e., when everything fits into memory.

The basic new "noun" in pandas is the **data frame**.

## What is a data frame

It's like a table, with rows and columns (e.g., as in SQL).  Except:
  - The rows can be indexed by something interesting (there is special support for labels like categorical and timeseries data).  This is especially useful when you have timeseries data with potentially missing data points.
  - Cells can store Python objects. (Like in SQL, columns are homogeneous.)
  - Instead of "NULL", the name for a non-existent value is "NA".  Unlike R, Python's data frames only support NAs in columns of some data types (basically: floating point numbers and 'objects') -- but this is mostly a non-issue (because it will "up-cast" integers to float64, etc.)
  
Pandas provides a "batteries-included" basic data analysis:
  - **Loading data:** `read_csv`, `read_table`, `read_sql`, and `read_html`
  - **Selection, filtering, and aggregation** (i.e., SQL-type operations): There's a special syntax for `SELECT`ing.  There's the `merge` method for `JOIN`ing.  There's also an easy syntax for what in SQL is a mouthful: Creating a new column whose value is computed from other column -- with the bonus that now the computations can use the full power of Python (though it might be faster if it didn't).
  - **"Pivot table" style aggregation**: If you're an Excel cognosceti, you may appreciate this.
  - **NA handling**: Like R's data frames, there is good support for transforming NA values with default values / averaging tricks / etc.
  - **Basic statistics:** e.g. `mean`, `median`, `max`, `min`, and the convenient `describe`.
  - **Plugging into more advanced analytics:** Okay, this isn't batteries included.  But still, it plays reasonably with `sklearn`.
  - **Visualization:** For instance `plot` and `hist`.
  
We'll go through a little on all of these in the context of an example.  To go through it, you must have the (output) data files from the HMDA "Project structure" example.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re

We're going to explore a dataset of mortgage insurance issued by the Federal Housing Authority (FHA).  The data is broken down by census tract and tells us how big of a player the FHA is in each tract (how many homes etc ...).

## Loading data (and basic statistics / visualization)


In [3]:
names =["State_Code", "County_Code", "Census_Tract_Number", "NUM_ALL", "NUM_FHA", "PCT_NUM_FHA", "AMT_ALL", "AMT_FHA", "PCT_AMT_FHA"]
df = pd.read_csv('./data/fha_by_tract.csv', names=names)  ## Loading a CSV file, without a header (so we have to provide field names)

df['GEOID'] = df['Census_Tract_Number']*100 + 10**6 * df['County_Code'] \
    + 10**9 * df['State_Code']   ## A computed field!
    
df = df.sort_values('State_Code')  # sorting data to make it easier to read
df.head()

,State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID
23999,1.0,49.0,9613.0,16,4,25.0000,2184,799,36.58420,1.049961e+09
55215,1.0,3.0,102.0,8,1,12.5000,774,76,9.81912,1.003010e+09
65492,1.0,27.0,NaN,1,0,0.0000,82,0,0.00000,NaN
45193,1.0,95.0,311.0,20,3,15.0000,1495,263,17.59200,1.095031e+09
33750,1.0,39.0,9618.0,14,3,21.4286,1243,333,26.79000,1.039962e+09


In [5]:
# Basic statistics and a histogram of the percentage of mortages 
# in each census tract insured by FHA
print( df['PCT_AMT_FHA'].describe() )
df['PCT_AMT_FHA'].hist(bins=50, alpha=0.5)

count    72035.000000
mean        29.703179
std         24.037779
min          0.000000
25%         10.780800
50%         24.753900
75%         44.207550
max        100.000000
Name: PCT_AMT_FHA, dtype: float64


In [7]:
# The above distribution looks a little skewed, let's look at it's log

# We can save off the data into a new column
df['LOG_AMT_ALL'] = np.log(df['AMT_ALL'])
print (df['LOG_AMT_ALL'].describe())

# We can use the apply function to transform data
df['AMT_ALL'].apply(np.log).hist(bins=50)

count    72035.000000
mean         8.169060
std          1.431749
min          0.693147
25%          7.346655
50%          8.335192
75%          9.176577
max         14.270319
Name: LOG_AMT_ALL, dtype: float64


## Selecting

In [8]:
# Selecting off a column
print (df['State_Code'].head())

# Selecting off multiple columns
print (df[['State_Code', 'County_Code']].head())

# programatically access column names
print ([col for col in df])

23999    1.0
55215    1.0
65492    1.0
45193    1.0
33750    1.0
Name: State_Code, dtype: float64
       State_Code  County_Code
23999         1.0         49.0
55215         1.0          3.0
65492         1.0         27.0
45193         1.0         95.0
33750         1.0         39.0
['State_Code', 'County_Code', 'Census_Tract_Number', 'NUM_ALL', 'NUM_FHA', 'PCT_NUM_FHA', 'AMT_ALL', 'AMT_FHA', 'PCT_AMT_FHA', 'GEOID', 'LOG_AMT_ALL']


## Filtering

This is a slightly fancied up version of Python's index notation.  When you write something like 

        df['column_name']==5

what's actually happening is that pandas creates a new, __binary__, data series indexed by the same indexing set as `df`.  You can combine such binary series using the term-wise `&` (`and`) and term-wise `|` (`or`) operations.  For instance:

        df['column_name2']==MD & ( df['column_name1']==5 | df['column_name2']==6 )

Now the `df[...]` notation is very flexible:
  - It accepts column names;
  - It accepts column numbers (so long as there is no ambiguity with column names..);
  - It accepsts _binary data series!_
  
This means that you can write

        df[ df['column_name2']==MD & ( df['column_name1']==5 | df['column_name1']==6 ) ]
   
for what you would write in SQL as

>         SELECT * FROM df WHERE
            column_name2='MD" AND (column_name1=5 OR column_name1=6)

In [9]:
# Selection returns a boolean array ..
print ((df['State_Code'] == 1).head())

# ... we can apply the usual boolean operators to it
print (((df['State_Code'] == 1) & (df['Census_Tract_Number'] == 9613)).head())

# pandas indices take boolean lists of the appropriate length
print (((df['State_Code'] == 1) & (df['Census_Tract_Number'] == 9613)).head())

23999    True
55215    True
65492    True
45193    True
33750    True
Name: State_Code, dtype: bool
23999     True
55215    False
65492    False
45193    False
33750    False
dtype: bool
23999     True
55215    False
65492    False
45193    False
33750    False
dtype: bool


## Join-ing

The analogue of a

>             
    SELECT * 
        FROM df1
        INNER JOIN df2 
        ON df1.field_name=df2.field_name;

is

    df_joined = df1.merge(df2, on='field_name')

You can also do left / right / outer joins, mix-and-match column names, etc.  For that consult the Pandas documentation. (The example below will do a left join..)

Of course, just looking at the distribution of insurance by census tract isn't interesting unless we know more about the census tract.

In [11]:
# Loading information about census tracts
df_geo = pd.read_csv('./data/2013_Gaz_tracts_national.tsv', sep='\t')

# And join it in
df_joined = df.merge(df_geo, on='GEOID', how='left')
df_joined.sort_values('AMT_ALL', ascending=False).head()

,State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID,LOG_AMT_ALL,USPS,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
72034,NaN,NaN,NaN,9477,1932,20.38620,1575871,331515,21.036900,NaN,14.270319,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64830,48.0,157.0,6731.01,2329,363,15.58610,578838,76312,13.183700,4.815767e+10,13.268778,TX,50922560.0,384718.0,19.661,0.149,29.744280,-95.815507
7795,6.0,73.0,170.30,574,58,10.10450,284965,31840,11.173300,6.073017e+09,12.560122,CA,33370884.0,1297044.0,12.885,0.501,33.036238,-117.126757
8857,6.0,75.0,615.00,357,4,1.12045,271648,2165,0.796987,6.075062e+09,12.512262,CA,1669698.0,439050.0,0.645,0.170,37.787726,-122.392389
8890,6.0,59.0,626.43,130,2,1.53846,223532,1484,0.663887,6.059063e+09,12.317310,CA,18821408.0,1549909.0,7.267,0.598,33.595088,-117.829038


## Aggregation

The analog of SQL's `GROUP BY` is

    grouped = df.groupby(['field_name1', ...])

The above is analogous to
>             
    SELECT * 
        FROM df
        GROUP BY df.field_name1, ...

Pandas is somewhat more flexible in how you can use grouping, not requiring you to specify an aggregation function up front.  A few examples are:


In [12]:
# This isn't a SQL-style 'GROUP BY'.
df_joined.groupby('USPS').first().head()

,State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID,LOG_AMT_ALL,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
USPS,,,,,,,,,,,,,,,,,
AK,2.0,122.0,9.00,15,3,20.0000,3797,644,16.9608,2.122001e+09,8.241967,120428818.0,13605012.0,46.498,5.253,59.708319,-151.545304
AL,1.0,49.0,9613.00,16,4,25.0000,2184,799,36.5842,1.049961e+09,7.688913,91371430.0,86324.0,35.279,0.033,34.457495,-85.635214
AR,5.0,25.0,9702.00,14,4,28.5714,1645,535,32.5228,5.025970e+09,7.405496,812912293.0,1721231.0,313.867,0.665,33.814492,-92.147621
AZ,4.0,19.0,41.15,34,22,64.7059,3284,2264,68.9403,4.019004e+09,8.096817,3168111.0,0.0,1.223,0.000,32.143460,-110.918635
CA,6.0,37.0,4029.04,32,15,46.8750,4976,3229,64.8915,6.037403e+09,8.512382,1903988.0,0.0,0.735,0.000,34.026072,-117.736203


In [13]:
# This is the analog of
# SELECT USPS, SUM(AMT_FHA), SUM(AMT_ALL), ... FROM df GROUP BY USPS;
df_by_state = df_joined[['USPS', 'AMT_FHA', 'AMT_ALL', 'NUM_FHA', 'NUM_ALL']].groupby('USPS').sum()

df_by_state['PCT_AMT_FHA'] = 100.0 * df_by_state['AMT_FHA']  / df_by_state['AMT_ALL']
df_by_state['PCT_NUM_FHA'] = 100.0 * df_by_state['NUM_FHA']  / df_by_state['NUM_ALL']

# This sure looks different than the census-tract level histogram!
df_by_state['PCT_AMT_FHA'].hist(bins=20)

## NA handling


When you read in a CSV file / SQL data base there are often "NA" (or "null", "None", etc.) values.  The CSV reader has a special field for specifying how this is denoted, and SQL has the built-in notion of NULL.  Pandas provides some tools for working with these -- they are generally similar too (and a little bit worse than) `R`

- `isnull` / `notnull`: Testing for null-ness e.g., 
>       
        df['column_name'].isnull()
        
   returns a Boolean series
- `fillna`: Replacing null values by something else, e.g.,
>         
        df['column_name'].fillna(0)             # Fills constant value, here 0
        df['column_name'].fillna(method='ffill')  # Fill forwards
        df['column_name'].fillna(method='bfill', limit=5)  # Fill backwards, at most 5
        
    At least by default, this is *not in place* -- that is, it creates a new series and does not change the original one.

- `interpolate`: Replacing null values by (linear, or quadratic, or...) interpolation.  There is support for indexing by times (not necessarily equally spaced), etc. in the documentation.  The most basic usage is
>        
        df['column_name'].interpolate()
    
    As above, this is not in place.


For more details: http://pandas.pydata.org/pandas-docs/stable/missing_data.html